In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import f1_score
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
dset_idx = 0

In [4]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [5]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["figo_stage"]
cat_cols = ["race", "ethnicity"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/survival_data.tsv", event_code)

In [6]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="n")
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
        .pipe(prep.cols_to_front, ["sample_name", "figo_num"])
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')
print(filtered_survival_df.shape)
# filtered_survival_df.head()

(255, 12)


# Load normalized matrisome count data

In [7]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[dset_idx]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(255, 1009)


# Join survival and count data

In [8]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(255, 1019)


# Optimize model

In [9]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

## Get baselines

In [10]:
rand.seed(seed)

label_value_counts_df = (
    pd.DataFrame(y_df.figo_num.value_counts()).reset_index()
        .rename(columns={"index": "label", "figo_num": "n"})
        .sort_values("n", ascending=False)
)

most_frequent_label = label_value_counts_df.label[0]
most_frequent_baseline = f1_score(y_df.values.squeeze(), np.repeat(most_frequent_label, y_df.shape[0]), average="weighted")

mc_baseline = opt.mc_classification_baseline(
    y=y_df.values.squeeze(),
    labels=label_value_counts_df.label.values,
    weights=label_value_counts_df.n.values / label_value_counts_df.n.values.sum(),
    metric=lambda y, yhat: f1_score(y, yhat, average="weighted"),
    n=1001
)

print(f"Most frequent baseline: {most_frequent_baseline}")
print(f"Monte Carlo baseline: {mc_baseline.mean()}")

Most frequent baseline: 0.3665158371040725
Monte Carlo baseline: 0.3668668294440783


## SMBO

In [11]:
matrisome_genes = norm_filtered_matrisome_counts_t_df.columns[1:]

def objective(h_params, X, y, penalty_default, scoring_default, r, verbose=True):
    if verbose:
        print(h_params)

    model = make_pipeline(
        ColumnTransformer([
            ("standard", StandardScaler(), ["age_at_diagnosis"] + list(matrisome_genes))
        ], remainder="passthrough"),
        LogisticRegression(
            C=h_params[0],
            class_weight=h_params[1],
            solver=h_params[2],
            penalty=penalty_default,
            n_jobs=-1,
            random_state=r
        )
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring=scoring_default
    ))

In [12]:
l1_space = [
    Real(1e-1, 1e1, name="C"),
    Categorical(["balanced", None], name="class_weight"),
    Categorical(["liblinear", "saga"], name="solver")
]

l2_space = [
    Real(1e-1, 1e1, name="C"),
    Categorical(["balanced", None], name="class_weight"),
    Categorical(["newton-cg", "lbfgs", "sag", "saga"], name="solver")
]

no_penalty_space = [
    Real(1e-1, 1e1, name="C"),
    Categorical(["balanced", None], name="class_weight"),
    Categorical(["newton-cg", "lbfgs", "sag", "saga"], name="solver")
]

n_initial = 10 * len(l1_space)
n_calls = 50 * len(l1_space)

In [13]:
penalty_default = "l2"
scoring_default = "f1_weighted"
space = l2_space
callback_file = f"{unified_dsets[dset_idx]}_opt_lr_h_params_{penalty_default}_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass

res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, penalty_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls=n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n=5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[7.15825767317995, 'balanced', 'sag']
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.2463
Function value obtained: -0.4428
Current minimum: -0.4428
Iteration No: 2 started. Evaluating function at random point.
[7.219588070532254, 'balanced', 'sag']
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.9840
Function value obtained: -0.4568
Current minimum: -0.4568
Iteration No: 3 started. Evaluating function at random point.
[4.1681512898192885, None, 'lbfgs']
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.7731
Function value obtained: -0.5045
Current minimum: -0.5045
Iteration No: 4 started. Evaluating function at random point.
[4.0700738109459165, None, 'newton-cg']
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.7066
Function value obtained: -0.5078
Current minimum: -0.5078
Iteration No: 5 started. Evaluating function at random point.
[2.52

KeyboardInterrupt: 

In [ ]:
penalty_default = "l1"
scoring_default = "f1_weighted"
space = l1_space
callback_file = f"{unified_dsets[dset_idx]}_opt_lr_h_params_{penalty_default}_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass

res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, penalty_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls=n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n=5, sep="\t")
)

In [ ]:
penalty_default = "none"
scoring_default = "f1_weighted"
space = no_penalty_space
callback_file = f"{unified_dsets[dset_idx]}_opt_lr_h_params_{penalty_default}_{scoring_default}.tsv"

try:
    os.remove(callback_file)
except OSError:
    pass

res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, penalty_default, scoring_default, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls=n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n=5, sep="\t")
)